# Section 3: Pandas

Having explored NumPy in a previous section, it is time to get to know the other workhorse of data science in Python: pandas. The pandas library in Python really does a lot to make working with data--and importing, cleaning, and organizing it--so much easier that it is hard to imagine doing data science in Python without it.

But it was not always this way. Wes McKinney developed the library out of necessity in 2008 while at AQR Capital Management in order to have a better tool for dealing with data analysis. The library has since taken off as an open-source software project that has become a mature and integral part of the data science ecosystem. (In fact, some examples in this section will be drawn from McKinney's book, *Python for Data Analysis*.)

The name 'pandas' actually has nothing to do with Chinese bears but rather comes from the term *panel data*, a form of multi-dimensional data involving measurements over time that comes out the econometrics and statistics community. Ironically, while panel data is a usable data structure in pandas, it is not generally used today and we will not examine it in this course. Instead, we will focus on the two most widely used data structures in pandas: `Series` and `DataFrame`s.

## Reminders about importing and documentation

Just as you imported NumPy undwither the alias ``np``, we will import Pandas under the alias ``pd``:

In [1]:
import pandas as pd

As with the NumPy convention, `pd` is an important and widely used convention in the data science world; we will use it here and we advise you to use it in your own coding.

As we progress through the next section, don't forget that IPython provides tab-completion feature and function documentation with the ``?`` character. If you don't understand anything about a function you see in this section, take a moment and read the documentation; it can help a great deal. As a reminder, to display the built-in pandas documentation, use this code:

```ipython
In [4]: pd?
```

Because it can be useful to lean about `Series` and `DataFrame`s in pandas a extension of `ndarray`s in NumPy, go ahead also import NumPy; you will want it for some of the examples later on:

In [2]:
import numpy as np

Now, on to pandas!

## Fundamental panda data structures

Both `Series` and `DataFrame`s are a lot like they `ndarray`s you encountered in the last section. They provide clean, efficent data storage and handling at the scales necessary for data science. What both of them provide that `ndarray`s lack, however, are essential data-science features like flexibility when dealing with missing data and the ability to label data. These capabilities (along with others) help make `Series` and `DataFrame`s essential to the "data munging" that make up so much of data science.

### `Series` objects in pandas

A pandas `Series` is a lot like an `ndarray` in NumPy: a one-dimensional array of indexed data.
You can create a simple Series from an array of data like this:

In [3]:
series_example = pd.Series([-0.5, 0.75, 1.0, -2])
series_example

0   -0.50
1    0.75
2    1.00
3   -2.00
dtype: float64

Similar to an `ndarray`, a `Series` upcasts entries to be of the same type of data (that `-2` integer in the original array became a `-2.00` float in the `Series`).

What is different from an `ndarray` is that the ``Series`` automatically wraps both a sequence of values and a sequence of indices. These are two separate objects within the `Seriers` object that can access with the ``values`` and ``index`` attributes.

Try accessing the ``values`` first; they are just a familiar NumPy array:

In [5]:
series_example.values

array([-0.5 ,  0.75,  1.  , -2.  ])

The ``index`` is also an array-like object:

In [6]:
series_example.index

RangeIndex(start=0, stop=4, step=1)

Just as with `ndarra`s, you can access specific data elements in a `Series` via the familiar Python square-bracket index notation and slicing:

In [7]:
series_example[1]

0.75

In [8]:
series_example[1:3]

1    0.75
2    1.00
dtype: float64

Despite a lot of similarities, pandas `Series` have an important distinction from NumPy `ndarrays`: whereas `ndarrays` have  *implicitly defined* integer indices (as do Python lists), pandas `Series` have *explicitly defined* indices. The best part is that you can set the index:

In [9]:
series_example2 = pd.Series([-0.5, 0.75, 1.0, -2], index=['a', 'b', 'c', 'd'])
series_example2

a   -0.50
b    0.75
c    1.00
d   -2.00
dtype: float64

These explicit indices work exactly the way you would expect them to:

In [11]:
series_example2['b']

0.75

In [ ]:
# Do explicit Series indices work *exactly* the way you might expect?
# Try slicing series_example2 using its explicit index and find out.


With explicit indices in the mix, a `Series` is basically a fixed-length, ordered dictionary in that it maps arbitrary typed index values to arbitrary typed data values. But like `ndarray`s these data are all of the same type, which is important. Just as the type-specific compiled code behind `ndarray` makes them more efficient than a Python lists for certain operations, the type information of pandas ``Series`` makes them much more efficient than Python dictionaries for certain operations.

But the connection between `Series` and dictionaries is nevertheless very real: you can construct a ``Series`` object directly from a Python dictionary:

In [18]:
population_dict = {'France': 65429495,
                   'Germany': 82408706,
                   'Russia': 143910127,
                   'Japan': 126922333}
population = pd.Series(population_dict)
population

France      65429495
Germany     82408706
Japan      126922333
Russia     143910127
dtype: int64

Did you see what happened there? The order of the keys `Russia` and `Japan` in the switched places between the order in which they were entered in `population_dict` and how they ended up in the `population` `Series` object. While Python dictionary keys have no order, `Series` keys are ordered.

So, at one level, you can interact with `Series` as you would with dictionaries:

In [14]:
population['Russia']

143910127

But you can also do powerful array-like operations with `Series` like slicing:

In [ ]:
# Try slicing on the population Series on your own.
# Would slicing be possible if Series keys were not ordered?


You can also add elements to a `Series` the way that you would to an `ndarray`. Try it in the code cell below:

In [ ]:
# Try running population['Albania'] = 2937590 (or another country of your choice)
# What order do the keys appear in when you run population? Is it what you expected?


Anoter useful `Series` feature (and definitely a difference from dictionaries) is that `Series` automatically aligns differently indexed data in arithmetic operations:

In [20]:
pop2 = pd.Series({'Spain': 46432074, 'France': 102321, 'Albania': 50532})
population + pop2

Albania     2988122.0
France     65531816.0
Germany           NaN
Japan             NaN
Russia            NaN
Spain             NaN
dtype: float64

Notice that in the case of Germany, Japan, Russia, and Spain (and Albania, depending on what you did in the previous exercise), the addition operation produced `NaN` (not a number) values. pandas does not treat missing values as `0`, but as `NaN` (and it can be helpful to think of arithmetic operations involving `NaN` as essentially `NaN`$ + x=$ `NaN`).

### `DataFrame` object in pandas

The other crucial data structure in pandas to get to know for data science is the `DataFrame`.
Like the ``Series`` object, ``DataFrame``s can be thought of either as generalizations of `ndarray`s (or as specializations of Python dictionaries).

Just as a ``Series`` is like a one-dimensional array with flexible indices, a ``DataFrame`` is like a two-dimensional array with both flexible row indices and flexible column names. Essentially, a `DataFrame` represents a rectangular table of data and contains an ordered collection of labeled columns, each of which can be a different value type (`string`, `int`, `float`, etc.).
The DataFrame has both a row and column index; in this way you can think of it as a dictionary of `Series`, all of which share the same index.

Let's take a look at how this works in practice. We will start by creating a `Series` called `area`:

In [22]:
area_dict = {'Albania': 28748,
             'France': 643801,
             'Germany': 357386,
             'Japan': 377972,
             'Russia': 17125200}
area = pd.Series(area_dict)
area

Albania       28748
France       643801
Germany      357386
Japan        377972
Russia     17125200
dtype: int64

Now you can combine this with the `population` `Series` you created earlier by using a dictionary to construct a single two-dimensional table containing data from both `Series`:

In [57]:
countries = pd.DataFrame({'Population': population, 'Area': area})
countries

,Area,Population
Albania,28748,2937590
France,643801,65429495
Germany,357386,82408706
Japan,377972,126922333
Russia,17125200,143910127


As with `Series`, note that `DataFrame`s also automatically order indices (in this case, the column indices `Area` and `Population`).

So far we have combined dictionaries together to compose a `DataFrame` (which has given our `DataFrame` a row-centric feel), but you can also create `DataFrame`s in a column-wise fashion. Consider adding a `Capital` column using our reliable old array-analog, a list:

In [58]:
countries['Capital'] = ['Tirana', 'Paris', 'Berlin', 'Tokyo', 'Moscow']
countries

,Area,Population,Capital
Albania,28748,2937590,Tirana
France,643801,65429495,Paris
Germany,357386,82408706,Berlin
Japan,377972,126922333,Tokyo
Russia,17125200,143910127,Moscow


As with `Series`, even though initial indices are ordered in `DataFrame`s, subsequent additions to a `DataFrame` stay in the ordered added. However, you can explicitly change the order of `DataFrame` column indices this way:

In [59]:
countries = countries[['Capital', 'Area', 'Population']]
countries

,Capital,Area,Population
Albania,Tirana,28748,2937590
France,Paris,643801,65429495
Germany,Berlin,357386,82408706
Japan,Tokyo,377972,126922333
Russia,Moscow,17125200,143910127


Commonly in a data science context, it is necessary to generate new columns of data from existing data sets. Because `DataFrame` columns behave like `Series`, you can do this is by performing operations on them as you would with `Series`:

In [60]:
countries['Population Density'] = countries['Population'] / countries['Area']
countries

C:\Users\jeppesen\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Capital,Area,Population,Population Density
Albania,Tirana,28748,2937590,102.184152
France,Paris,643801,65429495,101.629999
Germany,Berlin,357386,82408706,230.587393
Japan,Tokyo,377972,126922333,335.798242
Russia,Moscow,17125200,143910127,8.403413


Note: don't worry if IPython gives you a warning over this. The warning is IPython trying to be a little too helpful. The new column you created is an actual part of the `DataFrame` and not a copy of a slice.

We have stated before that `DataFrame`s are like dictionaries, and it's true. You can retrieve the contents of a column just as you would the value for a specific key in an ordinary dictionary:

In [40]:
countries['Area']

Albania       28748
France       643801
Germany      357386
Japan        377972
Russia     17125200
Name: Area, dtype: int64

What about using the row indices?

In [ ]:
# Now try accessing row data with a command like countries['Japan']


This returns an error: `DataFrame`s are dictionaries of `Series`, which are the columns. `DataFrame` rows often have heterogeneous data types, so different methods are necessary to access row data. For that, we use the `.loc` method:

In [41]:
countries.loc['Japan']

Capital                   Tokyo
Area                     377972
Population            126922333
Population Density      335.798
Name: Japan, dtype: object

Note that what `.loc` returns is an indexed object in its own right and you can access elements within it using familiar index syntax:

In [42]:
countries.loc['Japan']['Area']

377972

In [ ]:
# Can you think of a way to return the area of Japan without using .iloc?
# Hint: Try putting the column index first.
# Can you slice along these indices as well?


Sometimes it is helpful in data science projects to add a column to a `DataFrame` without assigning values to it:

In [61]:
countries['Debt-to-GDP Ratio'] = np.nan
countries

C:\Users\jeppesen\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Capital,Area,Population,Population Density,Debt-to-GDP Ratio
Albania,Tirana,28748,2937590,102.184152,NaN
France,Paris,643801,65429495,101.629999,NaN
Germany,Berlin,357386,82408706,230.587393,NaN
Japan,Tokyo,377972,126922333,335.798242,NaN
Russia,Moscow,17125200,143910127,8.403413,NaN


Again, you can disregard the warning (if it triggers) about adding the column this way.

You can also add columns to a `DataFrame` that do not have the same number of rows as the `DataFrame`:

In [62]:
debt = pd.Series([0.19, 2.36], index=['Russia', 'Japan'])
countries['Debt-to-GDP Ratio'] = debt
countries

C:\Users\jeppesen\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Capital,Area,Population,Population Density,Debt-to-GDP Ratio
Albania,Tirana,28748,2937590,102.184152,NaN
France,Paris,643801,65429495,101.629999,NaN
Germany,Berlin,357386,82408706,230.587393,NaN
Japan,Tokyo,377972,126922333,335.798242,2.36
Russia,Moscow,17125200,143910127,8.403413,0.19


You can use the `del` command to delete a column from a `DataFrame`:

In [63]:
del countries['Capital']
countries

,Area,Population,Population Density,Debt-to-GDP Ratio
Albania,28748,2937590,102.184152,NaN
France,643801,65429495,101.629999,NaN
Germany,357386,82408706,230.587393,NaN
Japan,377972,126922333,335.798242,2.36
Russia,17125200,143910127,8.403413,0.19


In addition to their dictionary-like behavior, `DataFrames` also behave like two-dimensional arrays. For example, it can be useful at times when working with a `DataFrame` to transpose it:

In [64]:
countries.T

,Albania,France,Germany,Japan,Russia
Area,2.874800e+04,6.438010e+05,3.573860e+05,3.779720e+05,1.712520e+07
Population,2.937590e+06,6.542950e+07,8.240871e+07,1.269223e+08,1.439101e+08
Population Density,1.021842e+02,1.016300e+02,2.305874e+02,3.357982e+02,8.403413e+00
Debt-to-GDP Ratio,NaN,NaN,NaN,2.360000e+00,1.900000e-01


Again, note that `DataFrame` columns are `Series` and thus the data types must consistent, hence the upcasting to floating-point numbers. **If there had been strings in this `DataFrame`, everything would have been upcast to strings.** Use caution when transposing `DataFrame`s.

#### From a two-dimensional NumPy array

Given a two-dimensional array of data, we can create a ``DataFrame`` with any specified column and index names.
If omitted, an integer index will be used for each:

In [27]:
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

,foo,bar
a,0.865257,0.213169
b,0.442759,0.108267
c,0.047110,0.905718


## Manipulating data in pandas

A huge part of data science is manipulating data in order to analyze it. (One rule of thumb is that 80% of any data science project will be concerned with cleaning and organizing the data for the project.) So it makes sense to lear the tools that pandas provides for handling data in `Series` and especially `DataFrame`s. Because both of those data structures are ordered, let's first start by taking a closer look at what gives them their structure: the `Index`.

### Index objects in pandas

Both ``Series`` and ``DataFrame``s in pandas have explicit indices that enable you to reference and modify data in them. These indices are actually objects themselves. The ``Index`` object can be thought of as both an immutable array or as fixed-size set. 

It's worth the time to get to know the properties of the `Index` object. Let's return to an example from earlier in the section to examine these properties.

In [66]:
series_example = pd.Series([-0.5, 0.75, 1.0, -2], index=['a', 'b', 'c', 'd'])
ind = series_example.index
ind

Index(['a', 'b', 'c', 'd'], dtype='object')

The ``Index`` works a lot like an array. we have already seen how to use standard Python indexing notation to retrieve values or slices:

In [67]:
ind[1]

'b'

In [68]:
ind[::2]

Index(['a', 'c'], dtype='object')

But ``Index`` objects are immutable; you cannot be modified via the normal means:

In [69]:
ind[1] = 0

TypeError: Index does not support mutable operations

This immutability is a good thing: it makes it safer to share indices between multiple ``Series`` or ``DataFrame``s without the potential for problems arising from inadvertent index modification.

In addition to being array-like, a Index also behaves like a fixed-size set, including following many of the conventions used by Python's built-in ``set`` data structure, so that unions, intersections, differences, and other combinations can be computed in a familiar way. Let's play around with this to see it in action.

In [35]:
ind_odd = pd.Index([1, 3, 5, 7, 9])
ind_prime = pd.Index([2, 3, 5, 7, 11])

In the code cell below, try out the intersection (`ind_odd & ind_prime`), union (`ind_odd | ind_prime`), and the symmetric difference (`ind_odd ^ ind_prime`) of `ind_odd` and `ind_prime`.

These operations may also be accessed via object methods, for example ``ind_odd.intersection(ind_prime)``. Below is a table listing some useful `Index` methods and properties.

| **Method**     | **Description**                                                                           |
|:---------------|:------------------------------------------------------------------------------------------|
| [`append`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html)       | Concatenate with additional `Index` objects, producing a new `Index`                      |
| [`diff`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.diff.html)         | Compute set difference as an Index                                                        |
| [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html)         | Compute new `Index` by deleting passed values                                             |
| [`insert`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.insert.html)       | Compute new `Index` by inserting element at index `i`                                     |
| [`is_monotonic`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.is_monotonic.html) | Returns `True` if each element is greater than or equal to the previous element           |
| [`is_unique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.is_unique.html)    | Returns `True` if the Index has no duplicate values                                       |
| [`isin`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)         | Compute boolean array indicating whether each value is contained in the passed collection |
| [`unique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html)       | Compute the array of unique values in order of appearance                                         |

### Data Selection in Series

As a refresher, a ``Series`` object acts in many ways like both a one-dimensional `ndarray` and a standard Python dictionary.

Like a dictionary, the ``Series`` object provides a mapping from a collection of arbitrary keys to a collection of arbitrary values. Back to an old example:

In [72]:
series_example2 = pd.Series([-0.5, 0.75, 1.0, -2], index=['a', 'b', 'c', 'd'])
series_example2

a   -0.50
b    0.75
c    1.00
d   -2.00
dtype: float64

In [74]:
series_example2['b']

0.75

You can also examine the keys/indices and values using dictionary-like Python tools:

In [75]:
'a' in series_example2

True

In [76]:
series_example2.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [77]:
list(series_example2.items())

[('a', -0.5), ('b', 0.75), ('c', 1.0), ('d', -2.0)]

As with dictionaries, you can extend a dictionary by assigning to a new key, you can extend a ``Series`` by assigning to a new index value:

In [78]:
series_example2['e'] = 1.25
series_example2

a   -0.50
b    0.75
c    1.00
d   -2.00
e    1.25
dtype: float64

#### Series as one-dimensional array

Because ``Series`` also provide array-style functionality, you can use the NumPy techniques we looked at a previous section like slices, masking, and fancy indexing:

In [79]:
# Slicing using the explicit index
series_example2['a':'c']

a   -0.50
b    0.75
c    1.00
dtype: float64

In [80]:
# Slicing using the implicit integer index
series_example2[0:2]

a   -0.50
b    0.75
dtype: float64

In [82]:
# Masking
series_example2[(series_example2 > -1) & (series_example2 < 0.8)]

a   -0.50
b    0.75
dtype: float64

In [84]:
# Fancy indexing
series_example2[['a', 'e']]

a   -0.50
e    1.25
dtype: float64

One note to avoid confusion. When slicing with an explicit index (i.e., ``series_example2['a':'c']``), the final index is **included** in the slice; when slicing with an implicit index (i.e., ``series_example2[0:2]``), the final index is **excluded** from the slice.

#### Indexers: `loc` and `iloc`

A great thing about pandas is that you can use a lot different things for your explicit indices. A potentially confusing thing about pandas is that you can use a lot different things for your explicit indices, including integers. To avoid confusion between integer indices that you might supply and those implicit integer indices that pandas generates, pandas provides special *indexer* attributes that explicitly expose certain indexing schemes.

(A technical note: These are not functional methods; they are attributes that expose a particular slicing interface to the data in the ``Series``.)

The ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [86]:
series_example2.loc['a']

-0.5

In [88]:
series_example2.loc['a':'c']

a   -0.50
b    0.75
c    1.00
dtype: float64

The ``iloc`` attribute enables indexing and slicing using the implicit, Python-style index:

In [90]:
series_example2.iloc[0]

-0.5

In [91]:
series_example2.iloc[0:2]

a   -0.50
b    0.75
dtype: float64

A guiding principle of the Python language is the idea that "explicit is better than implicit." Professional code will generally use explicit indexing with ``loc`` and ``iloc`` and you should as well in order to make your code clean and readable.

### Data selection in DataFrames

``DataFrame``s also exhibit dual behavior, acting both like a two-dimensional `ndarray` and like a dictionary of ``Series``  sharing the same index.

#### DataFrame as dictionary of Series

Let's return to our earlier example of countries' areas and populations in order to examine `DataFrame`s as a dictionary of `Series`.

In [92]:
area = pd.Series({'Albania': 28748,
                  'France': 643801,
                  'Germany': 357386,
                  'Japan': 377972,
                  'Russia': 17125200})
population = pd.Series ({'Albania': 2937590,
                         'France': 65429495,
                         'Germany': 82408706,
                         'Russia': 143910127,
                         'Japan': 126922333})
countries = pd.DataFrame({'Area': area, 'Population': population})
countries

,Area,Population
Albania,28748,2937590
France,643801,65429495
Germany,357386,82408706
Japan,377972,126922333
Russia,17125200,143910127


You can access the individual ``Series`` that make up the columns of a ``DataFrame`` via dictionary-style indexing of the column name:

In [94]:
countries['Area']

Albania       28748
France       643801
Germany      357386
Japan        377972
Russia     17125200
Name: Area, dtype: int64

An you can use dictionary-style syntax can also be used to modify `DataFrame`s, such as by adding a new column:

In [95]:
countries['Population Density'] = countries['Population'] / countries['Area']
countries

,Area,Population,Population Density
Albania,28748,2937590,102.184152
France,643801,65429495,101.629999
Germany,357386,82408706,230.587393
Japan,377972,126922333,335.798242
Russia,17125200,143910127,8.403413


#### DataFrame as two-dimensional array

You can also think of ``DataFrame``s as two-dimensional arrays. You can examine the raw data in the `DataFrame`/data array using the ``values`` attribute:

In [96]:
countries.values

array([[  2.87480000e+04,   2.93759000e+06,   1.02184152e+02],
       [  6.43801000e+05,   6.54294950e+07,   1.01629999e+02],
       [  3.57386000e+05,   8.24087060e+07,   2.30587393e+02],
       [  3.77972000e+05,   1.26922333e+08,   3.35798242e+02],
       [  1.71252000e+07,   1.43910127e+08,   8.40341292e+00]])

Viewed thsi way it makes sense that we can transpose the rows and columns of a `DataFrame` the same way we would an array:

In [97]:
countries.T

,Albania,France,Germany,Japan,Russia
Area,2.874800e+04,6.438010e+05,3.573860e+05,3.779720e+05,1.712520e+07
Population,2.937590e+06,6.542950e+07,8.240871e+07,1.269223e+08,1.439101e+08
Population Density,1.021842e+02,1.016300e+02,2.305874e+02,3.357982e+02,8.403413e+00


`DataFrame`s also uses the ``loc`` and ``iloc`` indexers. With ``iloc``, you can index the underlying array as if it were an `ndarray` but with the ``DataFrame`` index and column labels maintained in the result:

In [98]:
countries.iloc[:3, :2]

,Area,Population
Albania,28748,2937590
France,643801,65429495
Germany,357386,82408706


``loc`` also permits array-like slicing but using the explicit index and column names:

In [100]:
countries.loc[:'Germany', :'Population']

,Area,Population
Albania,28748,2937590
France,643801,65429495
Germany,357386,82408706


You can also use array-like techniques such as masking and fancing indexing with `loc`.

In [101]:
# Can you think of how to combine masking and fancy indexing in one line?
# Your masking could be somthing like countries['Population Density'] > 200
# Your fancy indexing could be something like ['Population', 'Population Density']
# Be sure to put the the masking and fancy indexing inside the square brackets: countries.loc[]


#### Indexing conventions

In practice in the world of data science (and pandas more generally), *indexing* refers to columns while *slicing* refers to rows:

In [102]:
countries['France':'Japan']

,Area,Population,Population Density
France,643801,65429495,101.629999
Germany,357386,82408706,230.587393
Japan,377972,126922333,335.798242


Such slices can also refer to rows by number rather than by index:

In [103]:
countries[1:3]

,Area,Population,Population Density
France,643801,65429495,101.629999
Germany,357386,82408706,230.587393


Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

In [104]:
countries[countries['Population Density'] > 200]

,Area,Population,Population Density
Germany,357386,82408706,230.587393
Japan,377972,126922333,335.798242


These two conventions are syntactically similar to those on a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.

# Operating on Data in Pandas

As you begin to work in data science, operating on data is imperative. It is the very heart of data science. Another aspect of pandas that makes it a compelling tool for many data scientists is pandas' capability to perform efficient element-wise operations on data. pandas builds on ufuncs from NumPy to supply theses capabilities and then extends them to provide additional power for data manipulation:
 - For unary operations (such as negation and trigonometric functions), ufuncs in pandas **preserve index and column labels** in the output.
 - For binary operations (such as addition and multiplication), pandas automatically **aligns indices** when passing objects to  ufuncs.

These critical features of ufuncs in pandas mean that data retains its context when operated on and, more importantly still, drastically helps reduce errors when you combine data from multiple sources.

## Index Preservation

pandas is explicitly designed to work with NumPy. As a results, all NumPy ufuncs will work on Pandas ``Series`` and ``DataFrame`` objects.

We can see this more clearly if we create a simple ``Series`` and ``DataFrame`` of random numbers on which to operate. 

In [8]:
rng = np.random.RandomState(42)
ser_example = pd.Series(rng.randint(0, 10, 4))
ser_example

0    6
1    3
2    7
3    4
dtype: int32

Did you notice the NumPy function we used with the variable `rng`? By specifying a seed for the random-number generator, you get the same result each time. This can be useful trick when you need to produce psuedo-random output that also needs to be replicatable by others. (Go ahead and re-run the code cell above a couple of times to convince yourself that it produces the same output each time.)

In [9]:
df_example = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df_example

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Let's apply a ufunc to our example `Series`:

In [11]:
np.exp(ser_example)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

The same thing happens with a slightly more complex operation on our example `DataFrame`:

In [12]:
np.cos(df_example * np.pi / 4)

,A,B,C,D
0,-1.836970e-16,7.071068e-01,6.123234e-17,-1.836970e-16
1,7.071068e-01,-1.000000e+00,-7.071068e-01,7.071068e-01
2,7.071068e-01,6.123234e-17,-7.071068e-01,-1.000000e+00


Note that you can use all of the ufuncs we discussed in a previous section the same way.

## Index alignment

As mentioned above, when you perform a binary operation on two ``Series`` or ``DataFrame`` objects, pandas will align indices in the process of performing the operation. This is essential when working with incomplete data (and data is usually incomplete), but it is helpful to see this in action to better understand it.

### Index alignment with Series

For our first example, suppose we are combining two different data sources and find only the top five countries by *area* and the top five countries by *population*:

In [15]:
area = pd.Series({'Russia': 17075400, 'Canada':  9984670,
                  'USA': 9826675, 'China': 9598094, 
                  'Brazil': 8514877}, name='area')
population = pd.Series({'China': 1409517397, 'India': 1339180127,
                        'USA': 324459463, 'Indonesia': 322179605, 
                        'Brazil': 207652865}, name='population')

In [ ]:
# Now divide these to compute the population density


Your resulting array contains the **union** of indices of the two input arrays: seven countries in total. All of the countries in the array without an entry (because they lacked either area data or population data) are marked with the now familiar ``NaN``, or "Not a Number," designation.

Index matching works the same way built-in Python arithmetic expressions and missing values are filled in with `NaN`s. You can see this clearly by adding two `Series` that are slightly misaligned in their indices:

In [18]:
series1 = pd.Series([2, 4, 6], index=[0, 1, 2])
series2 = pd.Series([3, 5, 7], index=[1, 2, 3])
series1 + series2

0     NaN
1     7.0
2    11.0
3     NaN
dtype: float64

`NaN` values are not always convenient to work with; `NaN` combined with any other values results in `NaN`, which can be a pain, particulalry if you are combining multiple data sources with missing values. To help with this, pandas allows you to specify a default value to use for missing values in the operation. For example, calling `series1.add(series2)` is equivalent to calling `series1 + series2`, but you can supply the fill value:

In [19]:
series1.add(series2, fill_value=0)

0     2.0
1     7.0
2    11.0
3     7.0
dtype: float64

Much better!

### Index alignment with DataFrames

The same kind of alignment takes place in both dimension (columns and indices) when you perform operations on ``DataFrame``s.

In [20]:
df1 = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                   columns=list('AB'))
df1

,A,B
0,1,11
1,5,1


In [22]:
df2 = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                   columns=list('BAC'))
df2

,B,A,C
0,3,8,2
1,4,2,6
2,4,8,6


In [ ]:
# Add df1 and df2. Is the output what you expected?

Even though we passed the columns in a different order in `df2` than in `df1`, the indices were aligned correctly sorted in the resulting union of columns.

You can also use fill values for missing values with `Data Frame`s. In this example, let's fill the missing values with the mean of all values in `df1` (computed by first stacking the rows of `df1`):

In [24]:
fill = df1.stack().mean()
df1.add(df2, fill_value=fill)

,A,B,C
0,9.0,14.0,6.5
1,7.0,5.0,10.5
2,12.5,8.5,10.5


This table lists Python operators and their equivalent pandas object methods:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Operations between DataFrames and Series

Index and column alignment gets maintained in operations between a `DataFrame` and a `Series` as well. To see this, consider a common operation in data science, wherein we find the difference of a `DataFrame` and one of its rows. Because pandas inherits ufuncs from NumPy, pandas will compute the difference row-wise by default:

In [25]:
df3 = pd.DataFrame(rng.randint(10, size=(3, 4)), columns=list('WXYZ'))
df3

,W,X,Y,Z
0,1,3,8,1
1,9,8,9,4
2,1,3,6,7


In [26]:
df3 - df3.iloc[0]

,W,X,Y,Z
0,0,0,0,0
1,8,5,1,3
2,0,0,-2,6


But what if you need to operate column-wise? You can do this by using object methodsand specifying the ``axis`` keyword.

In [28]:
df3.subtract(df3['X'], axis=0)

,W,X,Y,Z
0,-2,0,5,-2
1,1,0,1,-4
2,-2,0,3,4


And when you do operations between `DataFrame`s and `Series` operations, you still get automatic index alignment:

In [29]:
halfrow = df3.iloc[0, ::2]
halfrow

W    1
Y    8
Name: 0, dtype: int32

Note that the output from that operation was transposed. That was so that we can subtract it from the `DataFrame`:

In [30]:
df3 - halfrow

,W,X,Y,Z
0,0.0,NaN,0.0,NaN
1,8.0,NaN,1.0,NaN
2,0.0,NaN,-2.0,NaN


Remember, pandas preserves and aligns indices and columns so preserve data context. This will be of huge help to you in our next section when we look at data cleaning and preparation.